<pre>
<b>
COMP-6721 | 2021-Winter
Project | Part-1

Pravesh Gupta | 40152506
Vikramjeet Singh | 
Manjot Kaur Dherdi | 
</b>
</pre>

# 1. Data Loading

## 1.1. Installing required python modules

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=8791b43b8cde719410e725d1359a1a716b735865b6ba038e74ce9f9986d149e0
  Stored in directory: c:\users\vkrmj\appdata\local\pip\cache\wheels\22\0b\40\fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


### 1.2. Dataset Description


Required Dataset Folder Heirarchy:
<pre>
{dataset_folder}
--images
----mask
-------{mask images}
----no_mask
-------{no mask images}
----not_person
-------{not person images}
</pre>

### 1.3. Defining Pytorch dataset Representation and data transaformers

In [15]:
from torchvision.datasets import VisionDataset
import pandas as pd
import os
from skimage import io as sk_io, transform as sk_transform
import numpy as np
from sklearn.utils import shuffle
from PIL import Image
    
class Rescale(object):
    def __init__(self, output_size, debug=False, export_path=None):
        assert isinstance(output_size, (int))
        self.output_size = output_size

    def __call__(self, img_data):
        img_arr = np.array(img_data)
        scale_factor = float(self.output_size)/img_arr.shape[0]
        img_arr = (sk_transform.rescale(img_arr, (scale_factor, scale_factor, 1))*255).astype(np.uint8)
        
        new_w = img_arr.shape[1]
        
        # Clipping or filling
        if new_w>self.output_size:
            mid = new_w//2
            new_w_start = mid-self.output_size//2
            new_w_end = mid+self.output_size//2
            
            if (new_w_end-new_w_start)<self.output_size:
                new_w_end += (self.output_size-(new_w_end-new_w_start))
            elif (new_w_end-new_w_start)>self.output_size:
                new_w_end -= ((new_w_end-new_w_start)-self.output_size)
            img_arr = img_arr[:, new_w_start:new_w_end]
        elif new_w<self.output_size:
            mid = new_w//2
            new_w_start = self.output_size//2-mid
            new_w_end = new_w_start+new_w
            filled_img_arr = np.zeros((self.output_size, self.output_size, img_arr.shape[2]), dtype=np.uint8)
            filled_img_arr[:, new_w_start:new_w_end] = img_arr[:, :]
            img_arr = filled_img_arr
        return Image.fromarray(img_arr)

### 1.4. Creating dataset and data loaders for train and test

In [16]:
def get_train_test_indices(dataset_targets):
    test_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        test_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    train_indices_map = {}
    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        train_indices_map[label] = {'indices': np.array([], dtype=np.int32), 'count': 0}

    targets = np.array(dataset.targets, dtype=np.int32)
    target_indices = np.where(targets!=None)[0]
    np.random.shuffle(target_indices)

    for i in target_indices:
        label = dataset.targets[i]
        if test_indices_map[label]['count']<100:
            test_indices_map[label]['indices'] = np.append(test_indices_map[label]['indices'], i)
            test_indices_map[label]['count']+=1
        elif train_indices_map[label]['count']<350:
            train_indices_map[label]['indices'] = np.append(train_indices_map[label]['indices'], i)
            train_indices_map[label]['count']+=1

    test_indices = np.array([], dtype=np.int32)
    train_indices = np.array([], dtype=np.int32)

    for class_name in dataset.class_to_idx:
        label = dataset.class_to_idx[class_name]
        label_test_indices = test_indices_map[label]['indices']
        test_indices = np.append(test_indices, label_test_indices)

        label_train_indices = train_indices_map[label]['indices']
        train_indices = np.append(train_indices, label_train_indices)
    return train_indices, test_indices

In [17]:
from torchvision.transforms import ToTensor, Compose, Normalize
import torch
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import ImageFolder

data_transform = Compose([
    Rescale(512)
    ,ToTensor()
#     ,Normalize(mean=torch.zeros((3)),std=torch.ones((3)))
])

dataset = ImageFolder(
    root='D:/Courses/COMP 6721 Applied AI/Project/Iteration1/Dataset/images/'
    ,transform=data_transform
)
print(f'Dataset: size: {len(dataset)}, class labels: {dataset.class_to_idx}')

train_indices, test_indices = get_train_test_indices(dataset.targets)
    
train_dataset = Subset(dataset, train_indices)
print(f'Train dataset: size: {len(train_dataset)}')

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
for data, labels in train_dataloader:
    print(f'Train X | y batch shapes : {data.shape, labels.shape}')
    break

test_dataset = Subset(dataset, test_indices)
print(f'Test dataset: size: {len(test_dataset)}')

test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)
for data, labels in test_dataloader:
    print(f'Test X | y batch shapes : {data.shape, labels.shape}')
    break

Dataset: size: 3091, class labels: {'mask': 0, 'no_mask': 1, 'not_person': 2}
Train dataset: size: 1050
Train X | y batch shapes : (torch.Size([10, 3, 512, 512]), torch.Size([10]))
Test dataset: size: 300
Test X | y batch shapes : (torch.Size([10, 3, 512, 512]), torch.Size([10]))


# 2. Data Modelling

In [18]:
from torch.nn import Module, Conv2d, MaxPool2d, Linear, ReLU

class ProjectModel(Module):
    def __init__(self):
        super(ProjectModel, self).__init__()
        self.conv1 = Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=2)
        self.pool1 = MaxPool2d(2)
        self.conv2 = Conv2d(in_channels=16, out_channels = 32, kernel_size=3)
        self.linear = Linear(in_features = 32*125*125, out_features=3)
        
    def forward(self, X):
        relu = ReLU(inplace=False)
        X = self.conv1(X)
        X = relu(X)
        X = self.pool1(X)
        X = self.conv2(X)
        X = relu(X)
        X = torch.flatten(X, 1)
        X = self.linear(X)
        return X

net = ProjectModel()

### 2.1. Integrity Test

In [19]:
from sklearn import metrics

# Model Unit test
for data, labels in train_dataloader:
    outputs = net(data)
    print(outputs)
    _, y_pred = torch.max(outputs.data, 1)
    print(y_pred)
    prob = torch.nn.functional.softmax(outputs, dim=1)
    print(prob)
    label_pred = torch.argmax(prob, dim=1)
    print(label_pred)
    acc = metrics.accuracy_score(labels, y_pred, normalize=True)
    break
print('Model test passed.')

tensor([[-0.0044, -0.0378, -0.0162],
        [ 0.0201, -0.0536,  0.0202],
        [ 0.0127, -0.0462,  0.0125],
        [ 0.0087, -0.0673,  0.0153],
        [ 0.0023, -0.0628,  0.0035],
        [-0.0025, -0.0476, -0.0018],
        [ 0.0090, -0.0878,  0.0113],
        [-0.0106, -0.0555,  0.0276],
        [ 0.0034, -0.0724,  0.0143],
        [-0.0046, -0.0490,  0.0060]], grad_fn=<AddmmBackward>)
tensor([0, 2, 0, 2, 2, 2, 2, 2, 2, 2])
tensor([[0.3384, 0.3272, 0.3344],
        [0.3414, 0.3172, 0.3414],
        [0.3398, 0.3204, 0.3398],
        [0.3409, 0.3160, 0.3431],
        [0.3404, 0.3189, 0.3408],
        [0.3382, 0.3233, 0.3385],
        [0.3436, 0.3119, 0.3444],
        [0.3339, 0.3192, 0.3469],
        [0.3404, 0.3155, 0.3441],
        [0.3370, 0.3224, 0.3406]], grad_fn=<SoftmaxBackward>)
tensor([0, 2, 0, 2, 2, 2, 2, 2, 2, 2])
Model test passed.


### 2.2. Training

In [22]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
import time

num_epoch = 5
lr = 0.01
momentum = 0.9

loss_evaluater = CrossEntropyLoss()
optimizer = SGD(net.parameters(), lr=lr, momentum=momentum)

t = time.time()
for i in range(num_epoch):
    train_acc_sum = 0.0
    batch_count = 0
    t_epoch = time.time()
    for data, labels in train_dataloader:
        X, y = data, labels
        optimizer.zero_grad()
        output = net(X)
        loss = loss_evaluater(output, y)
        loss.backward()
        optimizer.step()
        _, y_pred = torch.max(output.data, 1)
        acc = metrics.accuracy_score(y, y_pred)
        batch_count+=1
        train_acc_sum+=acc
    epoch_acc = (train_acc_sum/batch_count)
    seconds_passed = time.time()-t_epoch
    print(f'Epoch {i+1}: [time:{seconds_passed//60}m {seconds_passed%60}s, Accuracy: {epoch_acc*100}]')
seconds_passed = time.time()-t
print(f'Training time: {seconds_passed//60}m {seconds_passed%60}s')
print('=============================================')

t = time.time()
with torch.no_grad():
    acc_sum = 0.0
    batch_count = 0
    for data, labels in train_dataloader:
        X, y = data, labels
        output = net(X)
        _, y_pred = torch.max(output.data, 1)
        acc = metrics.accuracy_score(y, y_pred)
        acc_sum+=acc
        batch_count+=1
    train_acc = acc_sum/batch_count
    print(f'Training accuracy: {train_acc*100}')
seconds_passed = time.time()-t
print(f'Trainig evaluation time: {seconds_passed//60}m {seconds_passed%60}s')

Epoch 1: [time:0.0m 2.8794808387756348s, Accuracy: 30.0]
Training time: 0.0m 2.8794808387756348s
Training accuracy: 33.33333333333332
Trainig evaluation time: 4.0m 53.35770297050476s


### 2.3. Testing

In [ ]:
t = time.time()
acc_sum = 0.0
precision_sum = 0.0
recall_sum = 0.0
f1_sum = 0.0
batch_count = 0
for data, labels in test_dataloader:
    X, y = data, labels
    output = net(X)
    _, y_pred = torch.max(output.data, 1)
    acc = metrics.accuracy_score(y, y_pred)
    acc_sum+=acc
    precision=metrics.precision_score(y, y_pred, average='weighted')
    precision_sum+=precision
    recall=metrics.recall_score(y, y_pred, average='weighted')
    recall_sum+=recall
    f1=metrics.f1_score(y, y_pred, average='weighted')
    f1_sum+=f1
    batch_count+=1
test_acc = acc_sum/batch_count
print(f'Test accuracy: {test_acc*100}')
test_precision = precision_sum/batch_count
print(f'Test precision: {test_precision*100}')
test_recall = recall_sum/batch_count
print(f'Test recall: {test_recall*100}')
test_f1 = f1_sum/batch_count
print(f'Test f1 score: {test_f1*100}')
seconds_passed = time.time()-t
print(f'Test time: {seconds_passed//60}m {seconds_passed%60}s')

# 3. Model Evaluation

### 3.1. Accuracy

In [32]:
print(f'Accuracy for the model is {test_acc*100} %')

Accuracy for the model is 50.0 %


### 3.2 Precision

In [41]:
print(f'Precision for the model is {test_precision*100} %')

Precision for the model is 4.0 %


### 3.3 Recall

In [42]:
print(f'Recall for the model is {test_recall*100} %')

Recall for the model is 20.0 %


### 3.4 F1 Score

In [43]:
print(f'F1 score for the model is {test_f1*100} %')

F1 score for the model is 6.666666666666668 %
